Variación del número de qubits total del sistema y del peso w para el circuito altamente entrelazado

In [ ]:
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info import Statevector, Operator
from qiskit.quantum_info import DensityMatrix, partial_trace
from qiskit.quantum_info import schmidt_decomposition
from qiskit.result import marginal_counts
from qiskit.primitives import StatevectorSampler, StatevectorEstimator

from qiskit.primitives import Sampler
from qiskit.visualization import plot_histogram
from qiskit.visualization import array_to_latex
from qiskit.visualization import latex
from qiskit.visualization import plot_bloch_vector
from qiskit.visualization import plot_bloch_multivector
from qiskit.visualization import visualize_transition


from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp


import matplotlib.pyplot as plt
import matplotlib.ticker as tck
from scipy.linalg import logm, expm


from matplotlib import cm
from matplotlib.ticker import LinearLocator

In [ ]:
def lista(n):

    di={}

    lista=[]

    for i in range(2**n):
        lista1=[]
        lista2=[]
        s=0
        for j in range(n):
            if str(format(i,'b').zfill(n))[j]=='1':
                s+=1
        if s==2:
            for j in range(n):
                if str(format(i,'b').zfill(n))[j]=='0':
                    lista1+='I'
                elif str(format(i,'b').zfill(n))[j]=='1':
                    lista1+='Z'
            lista2+=lista1[0]
            for j in range(1,n):
                lista2[0]+=lista1[j]
            di[str(format(i,'b').zfill(n))]=lista2[0]


    for i in range(2**n):
        if str(format(i, 'b').zfill(n)) in di:
            lista += tuple((di[str(format(i,'b').zfill(n))],))

    return lista

In [ ]:
n=7 #número total de qubits

m=int(n*(n-1)/2) #número de uniones
l=[]
w=1
for i in range(m):
    t=()
    strg=lista(n)[i]
    t+=(strg,w)
    l.append(t)

hamiltonian = SparsePauliOp.from_list(l)

ansatz = QAOAAnsatz(hamiltonian, reps=1)
# Draw
ansatz.decompose(reps=3).draw("mpl")

In [ ]:
#Grupo A: nA qubits  que se miden al salir de la puerta 
#Grupo B: nB qubits que reaprovechamos para el siguiente circuito al salir de la puerta conjunta


number_of_phases =100
phases_beta= np.linspace(0, 2 * np.pi, number_of_phases)
phases_gamma=np.linspace(0, 2 * np.pi, number_of_phases)

w=np.linspace(0,8,number_of_phases)
n=np.arange(2,7,1) #número total de qubits, de 3 a 6 (3,4,5,6) qubits


################Funciones#####################################################
def S(rho):
        return -((np.matmul(rho,logm(rho))).trace())


#Número de Schmidt:############################################################

def sch(rho):
    sch=0
    #uno de los subsistemas es [0], osea un solo qubit:
    #Using this convention, setting qargs=[0] will partition the state as |q3q2q1>|q0>
    
    a=schmidt_decomposition(rho,list(range(nA)))
    
    for i in range(len(a)):
        if a[i][0]!=1e-10:
            sch+=1
    return sch


##############################################################################

y=np.zeros((len(w),len(n)))  #entropía total
y_A=np.zeros((len(w),len(n))) #entropía subsistema A
y_B=np.zeros((len(w),len(n))) #entropía subsistema B
ym=np.zeros((len(w),len(n))) #entropía mutua
yc_AB=np.zeros((len(w),len(n))) #entropía condicionada A sobre B
yc_BA=np.zeros((len(w),len(n))) #entropía condicionada B sobre A

for j in n:
    
    m=int(j*(j-1)/2) #número de uniones

    nA=1
    nB=int(j-nA)


    
    for i in w:
        l=[]

        
        for h in range(m):
            t=()
            strg=lista(int(j))[h]
            t+=(strg,i)
            l.append(t)

    
        hamiltonian = SparsePauliOp.from_list(l)
        
        
        hamiltonian = SparsePauliOp.from_list(l)
        
        ansatz = QAOAAnsatz(hamiltonian, reps=1)    
        
        circuit=ansatz.assign_parameters([0,0.5])   #beta= 0 y gamma =0.5

        rho=DensityMatrix(circuit)

        print('número de Schmidt',sch(rho),'para un peso', i, 'para un nº de qubits',j)
    
        rho_A=partial_trace(DensityMatrix(circuit), qargs=list(range(nA,nA+nB))) #Matriz reducida sistema A
        rho_B=partial_trace(DensityMatrix(circuit), qargs=list(range(nA))) #Matriz reducida sistema B

        '''
        circuit.measure_all()
    
        results = Sampler().run(circuit).result()
        statistics = results.quasi_dists[0].binary_probabilities()
        '''
############Entropía############################

        b=n.tolist().index(j)
        a=w.tolist().index(i)
        
        y[a][b]=np.real(S(rho))
        y_A[a][b]=np.real(S(rho_A))
        y_B[a][b]=np.real(S(rho_B))
    
########Entropía mutua###########################

        Sm=S(rho_A)+S(rho_B)-S(rho)

        ym[a][b]=np.real(Sm)
    
        #print(f' Entropía mutua {Sm:.3f}')

##########Entropía condicional###################

        Sc_AB= S(rho)-S(rho_B)# rho_0 sobre rho_1
        Sc_BA= S(rho)-S(rho_A) # rho_1 sobre rho_0

        yc_AB[a][b]=np.real(Sc_AB)
        yc_BA[a][b]=np.real(Sc_BA)




Representación de resultados

In [ ]:
%matplotlib widget 

x=np.array(n)
z=np.array(w) 

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

X,Z=np.meshgrid(x,z)
my_col = cm.cool(y_A/np.amax(y_A))

surf = ax.plot_surface(X, Z, y_A, facecolors = my_col,
                        linewidth=0, antialiased=False)

plt.xlabel('w')
plt.ylabel('n')

ax.view_init(elev=50., azim=170, roll=0)

In [ ]:
from matplotlib.colors import LogNorm

plt.clf()

z=np.array(n)
x=np.array(w) 

c = plt.pcolormesh(z, x, y_A, cmap ='Greens')
plt.colorbar(c)

plt.ylabel('Peso w de uno de los ZZ (el resto son 1)')
plt.xlabel('qubits')

plt.title('Entropía Subsistema A', fontweight ="bold")

In [ ]:
#para un subsistema A de dos qubits

plt.clf()


%matplotlib widget 

x=np.array(n)
z=np.array(w) 

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

X,Z=np.meshgrid(x,z)
my_col = cm.jet(y_A/np.amax(y_A))

my_col=cm.cool(y_A/np.amax(y_A))

surf = ax.plot_surface(X, Z, y_A, facecolors = my_col,
                        linewidth=0, antialiased=False)

#plt.xlabel('qubits totales')
plt.ylabel('Peso w (rad)')
plt.title('Entropía subsistema A')

ax.view_init(elev=0., azim=0, roll=0)

In [ ]:
#Para un subsitema A de 3 qubits

plt.clf()


%matplotlib widget 

x=np.array(n)
z=np.array(w) 

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

X,Z=np.meshgrid(x,z)
my_col = cm.jet(y/np.amax(y))

surf = ax.plot_surface(X, Z, y, facecolors = my_col,
                        linewidth=0, antialiased=False)

ax.view_init(elev=50., azim=170, roll=0)